In [1]:
import numpy as np
from scipy import stats 
import math
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

import sqlalchemy
from sqlalchemy import create_engine
from sqlalchemy.orm import Session, sessionmaker

In [2]:
engine = create_engine("postgresql:///music", echo=True)
Session = sessionmaker(bind=engine)
session = Session()
con = engine.connect()

2019-05-08 16:45:06,707 INFO sqlalchemy.engine.base.Engine select version()
2019-05-08 16:45:06,708 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 16:45:06,710 INFO sqlalchemy.engine.base.Engine select current_schema()
2019-05-08 16:45:06,711 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 16:45:06,712 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-05-08 16:45:06,713 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 16:45:06,714 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-05-08 16:45:06,715 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 16:45:06,716 INFO sqlalchemy.engine.base.Engine show standard_conforming_strings
2019-05-08 16:45:06,716 INFO sqlalchemy.engine.base.Engine {}


In [32]:
query = """
    SELECT count(score), label, avg(score)
    FROM labels
    JOIN reviews
    USING(reviewid)
    GROUP BY label
    ORDER BY count(score) DESC
    LIMIT 10
    """
result = con.execute(query).fetchall()
result

2019-05-08 16:55:06,635 INFO sqlalchemy.engine.base.Engine 
    SELECT count(score), label, avg(score)
    FROM labels
    JOIN reviews
    USING(reviewid)
    GROUP BY label
    ORDER BY count(score) DESC
    LIMIT 10
    
2019-05-08 16:55:06,636 INFO sqlalchemy.engine.base.Engine {}


[(420, 'self-released', 6.91761907225563),
 (272, 'drag city', 7.17022060997346),
 (268, 'sub pop', 7.11679107556815),
 (244, 'thrill jockey', 7.06188525625917),
 (239, 'merge', 7.29121340767609),
 (217, 'warp', 7.15023041980058),
 (211, 'domino', 7.35213271362522),
 (207, 'columbia', 6.98550727977845),
 (202, 'matador', 7.36138615277734),
 (168, 'warner bros.', 6.60297619019236)]

In [33]:
labels = [tup[1] for tup in result]

In [35]:
bnm_by_label = {}
for label in labels:
    query = f"""
    SELECT best_new_music
    FROM reviews
    JOIN labels
    USING (reviewid)
    WHERE label = '{label}'
    """
    result = con.execute(query).fetchall() 
    bnm_list = [tup[0] for tup in result]
    bnm_by_label[label]=bnm_list    

2019-05-08 16:57:46,093 INFO sqlalchemy.engine.base.Engine 
    SELECT best_new_music
    FROM reviews
    JOIN labels
    USING (reviewid)
    WHERE label = 'self-released'
    
2019-05-08 16:57:46,095 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 16:57:46,100 INFO sqlalchemy.engine.base.Engine 
    SELECT best_new_music
    FROM reviews
    JOIN labels
    USING (reviewid)
    WHERE label = 'drag city'
    
2019-05-08 16:57:46,101 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 16:57:46,106 INFO sqlalchemy.engine.base.Engine 
    SELECT best_new_music
    FROM reviews
    JOIN labels
    USING (reviewid)
    WHERE label = 'sub pop'
    
2019-05-08 16:57:46,107 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 16:57:46,112 INFO sqlalchemy.engine.base.Engine 
    SELECT best_new_music
    FROM reviews
    JOIN labels
    USING (reviewid)
    WHERE label = 'thrill jockey'
    
2019-05-08 16:57:46,112 INFO sqlalchemy.engine.base.Engine {}
2019-05-08 16:57:46,118 INFO sqlalchemy.engine

In [37]:
mu_pop = 0.05126950470287609
alpha = 0.1
bonferroni_alpha = alpha / len(labels)

In [39]:
tests_by_label = {}
for label in bnm_by_label:
    bnms = bnm_by_label[label]
    t_val, p_val = stats.ttest_1samp(bnms, mu_pop)
    tests_by_label[label] = {
        't_val' : t_val,
        't_crit' : stats.t.ppf(1-bonferroni_alpha, len(bnms) -1),
        'p_val' : p_val,
    }

In [67]:
test_results_by_label = {}
for label in tests_by_label:
    t_val = tests_by_label[label]['t_val']
    t_crit = tests_by_label[label]['t_crit']
    p_val = tests_by_label[label]['p_val']
    if np.abs(t_val) > np.abs(t_crit) and np.abs(p_val) < bonferroni_alpha:
        test_results_by_label[label] = f'This indicates ' \
        f'that the alternative hypothesis is accepted with a p-val of {p_val}.'
    else: 
        test_results_by_label[label] = 'We fail to reject the null hypothesis.'
        

In [68]:
test_results_by_label

{'self-released': 'We fail to reject the null hypothesis.',
 'drag city': 'We fail to reject the null hypothesis.',
 'sub pop': 'This indicates that the alternative hypothesis is accepted with a p-val of 0.0006908554616336897.',
 'thrill jockey': 'This indicates that the alternative hypothesis is accepted with a p-val of 9.361615618190903e-25.',
 'merge': 'We fail to reject the null hypothesis.',
 'warp': 'This indicates that the alternative hypothesis is accepted with a p-val of 0.009615855389874585.',
 'domino': 'This indicates that the alternative hypothesis is accepted with a p-val of 7.376468407991896e-06.',
 'columbia': 'We fail to reject the null hypothesis.',
 'matador': 'This indicates that the alternative hypothesis is accepted with a p-val of 0.005715176789920098.',
 'warner bros.': 'We fail to reject the null hypothesis.'}